In [12]:
from langchain_openai import ChatOpenAI

from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Any
from langchain_core.output_parsers import JsonOutputParser
from typing_extensions import TypedDict
from typing import List

In [ ]:
import os
import openai

os.environ["OPENAI_API_KEY"] = ""
client = openai.OpenAI()

## 추가 질문 생성 LLM

In [3]:
example_specification = """
1. 사건 개요
	•	의뢰인: 가해자 (10대)
	•	거주 지역: 경남 고성군
	•	사건 내용: 전동킥보드를 운전 중 과속으로 인해 교통사고 발생

2. 사고 정보
	•	교통수단: 전동킥보드
	•	피해자: 전동킥보드 운전자 또는 동승자
	•	피해자 수: 2명
	•	피해 상황: 전치 4주 이상 상해
	•	대중과실: 속도위반 (제한속도보다 시속 20km 이상 초과)
	•	뺑소니 여부: 아니오
	•	보험 가입 여부: 가입되어 있음

3. 법적 진행 상황
	•	합의 여부: 이미 합의 완료
	•	전과: 교통사고 전과 있음
	•	사건 경과: 경찰 또는 검찰에서 수사 중
	•	답변: 고소 대리

"""

In [4]:
question_generation_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 법률 사례 분석 전문가입니다. 제공된 법률 명세서를 바탕으로 사례를 더 잘 이해하기 위한 중요한 추가 질문 3가지를 생성하세요.
    제공된 법률 명세서에 이미 있는 내용은 질문하지 말아야 합니다. 
    
    질문은 다음과 같은 영역을 다루어야 합니다:
    1. 사건의 중요 날짜나 시간적 맥락
    2. 관련 문서나 증거
    3. 이해관계자와의 상호작용
    
    질문은 구체적이고 명확해야 하며, 법률 사례 해결에 직접적으로 도움이 되어야 합니다.
    정확히 3개의 질문만 생성하고, 문자열 형식으로 반환하세요.
    질문의 길이는 최대한 짧고 간략하게 표현하세요. 
    '요'를 사용한 존댓말 질문 문장으로 생성하세요.
    
    질문 생성 후에는 '답변 가능한 선에서 최대한 구체적으로 작성해주세요.'라는 말을 필수로 덧붙여줘야 합니다. 
    """),
    ("human", "다음 법률 명세서를 바탕으로 추가 질문 3가지를 생성해주세요: {specification}")
])


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
parser = StrOutputParser()
chain = question_generation_prompt | llm | parser
result = chain.invoke({"specification": example_specification})

In [5]:
result

'1. 사건 발생일과 사고가 발생한 시간대는 언제인지요?\n2. 사고 당시 전동킥보드의 속도를 측정할 수 있는 증거가 있는지요?\n3. 피해자와의 합의 과정에서 어떤 조건들이 논의되었는지요? \n\n답변 가능한 선에서 최대한 구체적으로 작성해주세요.'

In [6]:
example_question = '''
아니, 그러니까 제가 그 회사를 다닌 게 한… 3년쯤 됐어요. 근데 갑자기 지난달인가, 팀장이 바뀌더니 저보고 실적이 부족하다고 하더라고요. 근데 원래 저 성과 좋았거든요? 아니, 진짜예요. 작년에는 인센티브도 받았어요. 근데 갑자기 말이 바뀌어서… 아, 맞다, 해고 통지가 온 게 3월 15일이었나? 근데 그날 바로 나가라고 해서 진짜 어이없었죠. 아, 그리고 퇴직금도 원래 바로 줘야 하는 거 아닌가요? ‘경영 사정’ 어쩌고 하면서 미뤄지는데, 이거 불법 아니에요?
아, 그리고 중요한 게 하나 더 있는데… 해고 전 한 달 동안 팀장이 새로 왔는데, 그 사람이 저한테 막 불가능한 목표를 주고 압박을 엄청 했거든요. 그거 진짜 일부러 그런 거 같아요. 막 ‘이 성과 안 나오면 자리 보장 못 한다’ 이런 식으로요. 아니, 저만 그런 것도 아니고 동료들도 다 이상하다고 했어요. 아, 그러고 보니 사무실에서 나가라고 할 때 다른 직원들도 좀 이상한 눈으로 보던데… 아무튼 이게 정당한 해고인지 아닌지 모르겠는데, 뭔가 이상하잖아요? 저 어떻게 해야 해요?
'''

In [7]:
system = """
당신은 법률 사례 분석 전문가입니다. 
당신은 주어진 내용을 재작성해야 합니다. 
이것은 이후 보고서 내용에 추가되어야하기 때문에 법률적, 전문적 의미가 더 명확하고 드러나도록 Rewrite 하세요.
영어를 한국어로 번역하지 마세요.
"""

re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("user", "원본 질문: {question} \n 새로운 질문:",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": example_question})

'저는 해당 회사에서 근무한 지 약 3년이 되었습니다. 그러나 지난달 팀장이 교체된 이후, 저에게 실적 부족을 이유로 문제를 제기하였습니다. 사실, 저는 이전에 성과가 우수하여 작년에는 인센티브를 수령하기도 했습니다. 그런데 갑자기 상황이 바뀌었습니다. 해고 통지는 3월 15일에 받았고, 그날 즉시 퇴사하라는 통보를 받았습니다. 이 상황이 매우 황당했습니다. 또한, 퇴직금 지급이 지연되고 있는데, 이는 ‘경영 사정’이라는 이유로 미뤄지고 있습니다. 이러한 지연이 법적으로 문제가 되지 않는지 궁금합니다.\n\n더불어, 해고 전 한 달 동안 새로 부임한 팀장이 저에게 비현실적인 목표를 부여하며 극심한 압박을 가했습니다. 이는 고의적인 행위로 보이며, ‘이 성과가 나오지 않으면 직위 보장이 어렵다’는 식의 발언을 하였습니다. 저뿐만 아니라 동료들도 이러한 상황에 대해 이상하다고 느끼고 있습니다. 해고 통보를 받을 당시, 다른 직원들이 저를 이상한 눈으로 바라보았던 것도 기억납니다. 이 모든 상황을 종합적으로 고려할 때, 제 해고가 정당한지 의문이 듭니다. 이러한 상황에서 제가 어떤 조치를 취해야 할지 조언을 부탁드립니다.'

## 보고서 제작

In [8]:
report_prompt = ChatPromptTemplate.from_messages([
("system", """
당신은 법률 전문가로서 법률 보고서를 작성해야 합니다. 제공된 법률 명세서, 추가 질문에 대한 답변, 
그리고 기타 정보를 바탕으로 체계적이고 전문적인 법률 보고서를 작성하세요.

보고서는 다음 구조를 따라야 합니다:

# 보고서 제목

## 사건 개요
### 사건명
### 배경
### 주요 쟁점

## 법적 분석 및 검토 내용
### 관련 법률 조항 및 해석
### 유사 판례 및 선례

## 예상되는 법적 진행 과정

## 법적 의견 및 권장 조치
### 법적 조치 가능성 검토 결과
### 승소 가능성 (%)
### 예상 소송 비용
### 예상 소송 소요 기간

보고서는 객관적인 사실과 법적 분석을 바탕으로 작성되어야 하며, 법률 용어를 적절히 사용하면서도 
일반인이 이해할 수 있도록 작성하세요. 승소 가능성은 판례 및 법리 분석을 바탕으로 합리적인 근거와 
함께 제시해야 합니다.
"""),
("human", """
다음 정보를 바탕으로 법률 보고서를 작성해주세요:

## 법률 명세서:
{legal_specification}

## 추가 질문에 대한 답변:
{additional_responses}

## 기타 추가 정보:
{extra_information}
""")
])

In [9]:
example_answer = """
	1.	사건 발생일 및 시간대
	•	사건은 2024년 3월 10일 오후 7시 30분경에 발생하였습니다.
	2.	전동킥보드 속도 기록 및 CCTV 영상 여부
	•	사고 당시 전동킥보드의 속도 측정 기록은 없으며, 해당 지역의 CCTV 영상은 확보된 상태입니다.
	•	현재 영상 분석을 통해 사고 경위를 추가 확인 중입니다.
	3.	피해자와의 합의 조건
	•	피해자는 치료비 전액 보상과 일정 금액의 위자료 지급을 요구하였습니다.
	•	가해자는 이에 대해 일부 조정 가능성을 논의하며 합의를 진행 중입니다.
	•	합의 과정에서 추가 손해배상 문제 및 책임 범위에 대한 의견 조율이 필요합니다.
"""

example_extra = """
현재 사고에 대한 법적 책임을 명확히 하기 위해 CCTV 영상 분석 및 목격자 진술 확보가 진행 중입니다. 
또한, 피해자의 치료 경과에 따라 추가적인 합의 조정 가능성이 있으며, 변호사와 상담 후 최종 합의서를 작성할 예정입니다. 
전동킥보드 관련 교통법규 위반 여부도 검토 중이며, 이에 따른 과태료나 행정처분 가능성이 있을 수 있습니다. 
합의가 원만히 이루어지지 않을 경우, 법적 절차 진행 여부도 고려할 필요가 있습니다.
"""

In [10]:
report_chain = report_prompt | llm | StrOutputParser()
report = report_chain.invoke({
    'legal_specification' : example_specification, 
    'additional_responses' : example_answer, 
    'extra_information' : example_extra
})

# langchain

In [13]:
class CRAGState(TypedDict): 
    specification: str
    answer: str
    extra: str

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(CRAGState)


workflow.add_node("")

workflow.set_entry_point()